In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
"""
dash==2.0.0
dash-core-components==2.0.0
dash-html-components==2.0.0
dash-table==5.0.0
pandas==1.2.4
plotly==5.4.0
gunicorn==20.1.0
"""

In [ ]:

        """      html.Div(
            style={"display": "grid",
                   "gap": 50,
                   "gridTemplateColumns": "repeat(2,1fr)",
                   },
            children=[
                html.Div(
                    style={"grid-column": "span 2"},
                    children=[
                        dcc.Dropdown(
                            style={
                                "width": 700,
                                "margin": "0 auto",
                                "color": "#111111",
                            },
                            placeholder="Select a Country",
                            id="country",
                            options=[
                                {'label': country, 'value': country}
                                for country in dropdown_options
                            ],
                        ),
                        dcc.Graph(id="country-graph"),
                    ])
            ]
        ), """

In [11]:
import pandas as pd
from datetime import date
from datetime import timedelta
from datetime import datetime, timezone
import numpy as np
from math import floor, ceil

pd.options.mode.chained_assignment = None

today = date.today()
yesterday = today - timedelta(days=1)
yesterday = yesterday.strftime('%m-%d-%Y')
two_days_ago = today - timedelta(days=2)
two_days_ago = two_days_ago.strftime('%Y-%m-%d')
two_days_ago_url = today - timedelta(days=2)
two_days_ago_url = two_days_ago_url.strftime('%m-%d-%Y')
now_utc = datetime.now(timezone.utc).strftime('%H')
#now_here = datetime.now().strftime('%H')

if now_utc > '06':
    daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{yesterday}.csv?raw=true'
else:
    daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{two_days_ago_url}.csv?raw=true'


daily_df = pd.read_csv(daily_url)
countries_df = daily_df[["Country_Region", "Confirmed", "Deaths"]]

countries_df = (countries_df.groupby("Country_Region").sum(
).sort_values(by="Confirmed", ascending=False).reset_index())


pop_url = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv?raw=true'
pop_df = pd.read_csv(pop_url)
pop_df = pop_df[(pop_df['Province_State'].isnull())]
pop_df = pop_df[["Country_Region", "Population", "iso3"]]

continent_url = 'https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv?raw=true'
continent_df = pd.read_csv(continent_url)
continent_df = continent_df[["alpha-3", "region", "sub-region"]]
continent_df = continent_df.rename(columns={'alpha-3': 'iso3'})
pop_df = pd.merge(pop_df, continent_df, on="iso3")

countries_df = pd.merge(countries_df, pop_df, on="Country_Region")

daily_time_url = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
daily_time_df = pd.read_csv(daily_time_url)
daily_time_df = daily_time_df.drop(["Lat", "Long"], axis=1)

daily_time_df_AuCaCh = daily_time_df.dropna(subset=['Province/State'])
daily_time_df_AuCaCh = daily_time_df_AuCaCh.groupby("Country/Region").sum()
daily_time_df_AuCaCh = daily_time_df_AuCaCh.loc[[
    'Australia', 'Canada', 'China'], :]
daily_time_df_AuCaCh = daily_time_df_AuCaCh.reset_index()
daily_time_df = daily_time_df[(daily_time_df['Province/State'].isnull())]
daily_time_df = daily_time_df.drop(["Province/State"], axis=1)
daily_time_df = daily_time_df.append(daily_time_df_AuCaCh, ignore_index=True)
####################################################
will_be_merged_df = daily_time_df.rename(
    columns={'Country/Region': 'Country_Region'})
merged_df = pd.merge(countries_df, will_be_merged_df,
                     on="Country_Region", how='inner')  # go to line 195
new_df = countries_df.iloc[:, :7]


####
for i in range(15, len(merged_df.columns)):
    new_df[merged_df.columns[i]] = (
        (merged_df.iloc[:, i] - merged_df.iloc[:, i-7]) / (merged_df["Population"]/100000)).astype(int)

new_df = new_df.drop(['Confirmed', 'Deaths'], axis=1)

new_melted_df = new_df.melt(id_vars=["Country_Region", "region", "sub-region", "Population", "iso3"],
                            var_name="Date",
                            value_name="7_Days_Incidence_Rate")

new_melted_df = new_melted_df.sort_values(by="Country_Region")
new_melted_df = new_melted_df.rename(
    columns={'7_Days_Incidence_Rate': 'Incidence_Rate'})
new_melted_df = new_melted_df.rename(columns={'sub-region': 'subregion'})
new_melted_df = new_melted_df.rename(columns={'Country_Region': 'Country'})

new_melted_df["Date"] = pd.to_datetime(new_melted_df["Date"])
new_melted_df["Date"] = new_melted_df["Date"].dt.strftime('%Y-%m-%d')
new_melted_df = new_melted_df.sort_values(by=['Country', 'Date']).reset_index()
new_melted_df = new_melted_df.drop('index', 1)
for i in new_melted_df.index[new_melted_df['Incidence_Rate'] < 0]:
    new_melted_df['Incidence_Rate'].iloc[i] = new_melted_df['Incidence_Rate'].iloc[i-1]
new_melted_df
########################################################
vac_owid_url = 'https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations.csv?raw=true'
df_vac_owid_for_merge = pd.read_csv(vac_owid_url)
df_vac_owid_for_merge = df_vac_owid_for_merge.loc[df_vac_owid_for_merge["date"] >= '2021-05-01']
df_vac_owid_for_merge = df_vac_owid_for_merge[[
    "location", "iso_code", "date", "people_fully_vaccinated", "total_boosters"]]
df_vac_owid_for_merge = df_vac_owid_for_merge.rename(
    columns={'iso_code': 'iso3'})
df_vac_owid_for_merge = df_vac_owid_for_merge.rename(columns={'date': 'Date'})
df_vac_owid_for_merge
new_melted_sub_df = new_melted_df.loc[new_melted_df["Date"] >= '2021-05-01']
new_melted_sub_df 
all_merged_df = new_melted_sub_df.merge(
    df_vac_owid_for_merge, on=["iso3", "Date"])
all_merged_df
all_merged_df[["people_fully_vaccinated"]] = all_merged_df[[
    "people_fully_vaccinated"]].fillna(method='ffill')
all_merged_df[["people_fully_vaccinated"]] = all_merged_df[[
    "people_fully_vaccinated"]].fillna(method='bfill')
all_merged_df = all_merged_df.drop('location', 1)
all_merged_df['Population'] = all_merged_df['Population'].astype(int)
all_merged_df["Fully_vaccinated_percent"] = (
    all_merged_df["people_fully_vaccinated"] / (all_merged_df["Population"]))*100
#########################################################
all_continents = new_melted_df.subregion.unique()
all_merged_df

,Country,region,subregion,Population,iso3,Date,Incidence_Rate,people_fully_vaccinated,total_boosters,Fully_vaccinated_percent
0,Afghanistan,Asia,Southern Asia,38928341,AFG,2021-05-01,3,55624.0,NaN,0.142888
1,Afghanistan,Asia,Southern Asia,38928341,AFG,2021-05-02,3,55624.0,NaN,0.142888
2,Afghanistan,Asia,Southern Asia,38928341,AFG,2021-05-03,3,55624.0,NaN,0.142888
3,Afghanistan,Asia,Southern Asia,38928341,AFG,2021-05-04,3,55624.0,NaN,0.142888
4,Afghanistan,Asia,Southern Asia,38928341,AFG,2021-05-05,3,55624.0,NaN,0.142888
...,...,...,...,...,...,...,...,...,...,...
41803,Zimbabwe,Africa,Sub-Saharan Africa,14862927,ZWE,2021-12-12,189,2976965.0,NaN,20.029467
41804,Zimbabwe,Africa,Sub-Saharan Africa,14862927,ZWE,2021-12-13,189,2988363.0,NaN,20.106154
41805,Zimbabwe,Africa,Sub-Saharan Africa,14862927,ZWE,2021-12-14,204,2998631.0,NaN,20.175239
41806,Zimbabwe,Africa,Sub-Saharan Africa,14862927,ZWE,2021-12-15,182,3013378.0,NaN,20.274459


In [ ]:
#For the bubble map
from datetime import date
from datetime import timedelta
from datetime import datetime,timezone
#today = datetime.today().strftime('%d-%m-%Y')
today = date.today()
yesterday = today - timedelta(days = 1)
yesterday = yesterday.strftime('%m-%d-%Y') 
two_days_ago = today - timedelta(days=2)
two_days_ago = two_days_ago.strftime('%Y-%m-%d')
two_days_ago_url = today - timedelta(days=2)
two_days_ago_url = two_days_ago_url.strftime('%m-%d-%Y')
now_utc = datetime.now(timezone.utc).strftime('%H') 
now_here = datetime.now().strftime('%H') 

if now_utc > '06' : 
    daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{yesterday}.csv?raw=true'
else: 
    daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{two_days_ago_url}.csv?raw=true'
    
    

daily_df = pd.read_csv(daily_url)
daily_df.loc[daily_df["Country_Region"] == "Germany"]
now_utc > '00'


In [ ]:
pop_url = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv?raw=true'
pop_df= pd.read_csv(pop_url)
pop_df= pop_df[(pop_df['Province_State'].isnull())]
pop_df = pop_df[["Country_Region","Population","iso3"]]
pop_df

continent_url = 'https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv?raw=true'
continent_df = pd.read_csv(continent_url)
continent_df = continent_df[["alpha-3","region","sub-region"]]
continent_df = continent_df.rename(columns={'alpha-3': 'iso3'})
#pop_df = pd.merge(pop_df,continent_df, on="iso3")
pop_df
continent_df
#pop_df.compare(continent_df)

In [ ]:
# calculate incident rate 
#danach vaccine per country 
daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{yesterday}.csv?raw=true'
daily_df = pd.read_csv(daily_url)
countries_df = daily_df[["Country_Region", "Confirmed", "Deaths"]]

countries_df = (countries_df.groupby("Country_Region").sum(
).sort_values(by="Confirmed", ascending=False).reset_index())


pop_url = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv?raw=true'
pop_df= pd.read_csv(pop_url)
pop_df= pop_df[(pop_df['Province_State'].isnull())]
pop_df = pop_df[["Country_Region","Population","iso3"]]
pop_df

continent_url = 'https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv?raw=true'
continent_df = pd.read_csv(continent_url)
continent_df = continent_df[["alpha-3","region","sub-region"]]
continent_df = continent_df.rename(columns={'alpha-3': 'iso3'})
pop_df = pd.merge(pop_df,continent_df, on="iso3")
countries_df = pd.merge(countries_df, pop_df, on="Country_Region")

daily_time_url ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
daily_time_df = pd.read_csv(daily_time_url)
daily_time_df = daily_time_df.drop(["Lat","Long"], axis =1)

daily_time_df_AuCaCh = daily_time_df.dropna(subset=['Province/State'])
daily_time_df_AuCaCh = daily_time_df_AuCaCh.groupby("Country/Region").sum()
daily_time_df_AuCaCh = daily_time_df_AuCaCh.loc[['Australia','Canada','China'], :]
daily_time_df_AuCaCh = daily_time_df_AuCaCh.reset_index()
daily_time_df= daily_time_df[(daily_time_df['Province/State'].isnull())]
daily_time_df = daily_time_df.drop(["Province/State"], axis =1)

daily_time_df = daily_time_df.append(daily_time_df_AuCaCh, ignore_index=True)
daily_time_df = daily_time_df.iloc[:, list(range(1)) + list(range(-8, 0))]
daily_time_df["Cases_in_last_7_days"] = (daily_time_df.iloc[:, -1] - daily_time_df.iloc[:, -8])
daily_time_df["Yesterday_Confirmed"] =  (daily_time_df.iloc[:, -2] - daily_time_df.iloc[:, -3])
daily_time_df = daily_time_df.iloc[:, list(range(1)) + list(range(-2, 0))]
daily_time_df = daily_time_df.rename(columns={'Country/Region': 'Country_Region'})
countries_df = pd.merge(countries_df, daily_time_df,on="Country_Region", how='inner')
countries_df["7_Days_Incidence_Rate"] = (countries_df["Cases_in_last_7_days"]/ (countries_df["Population"]/100000))

vac_url = 'https://github.com/govex/COVID-19/blob/master/data_tables/vaccine_data/global_data/vaccine_data_global.csv?raw=true'
df_vaccine = pd.read_csv(vac_url)
df_vaccine = df_vaccine[(df_vaccine['Province_State'].isnull())]
df_vaccine = df_vaccine[["Country_Region","People_fully_vaccinated"]]

countries_df = pd.merge(countries_df, df_vaccine,on="Country_Region", how='inner')
countries_df = countries_df.sort_values(by="7_Days_Incidence_Rate", ascending=False)
countries_df['Population'] =countries_df['Population'].astype(int)
countries_df['7_Days_Incidence_Rate'] =countries_df['7_Days_Incidence_Rate'].astype(int)
countries_df



vac_owid_url ='https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations.csv?raw=true'
df_vac_owid = pd.read_csv(vac_owid_url)
df_vac_owid = df_vac_owid[["location","date","people_fully_vaccinated"]]
df_vac_owid = df_vac_owid.loc[df_vac_owid["date"] == two_days_ago]
df_vac_owid_south_korea = df_vac_owid.loc[df_vac_owid["location"] == "South Korea"]
df_vac_owid_south_korea = df_vac_owid_south_korea[["people_fully_vaccinated"]]



loc_korea_vac_cell = countries_df.loc[countries_df.index[countries_df["Country_Region"] == "Korea, South"], 'People_fully_vaccinated'].item()
if pd.isnull(loc_korea_vac_cell) is True:
    countries_df[countries_df["Country_Region"] == "Korea, South"]= countries_df[countries_df["Country_Region"] == "Korea, South"].replace(to_replace =np.nan,
                 value =df_vac_owid_south_korea.iat[0,0])
else:
    pass

countries_df[countries_df["Country_Region"] == "Korea, South"]
#count = countries_df['People_fully_vaccinated'].isna().sum()
#print(count)
countries_df
#countries_df[countries_df['Country_Region'=='China']]
countries_df[countries_df.isna().any(axis=1)]
countries_df = countries_df.dropna()
#countries_df.index[countries_df["Country_Region"] == "Korea, South"]
countries_df['People_fully_vaccinated'] =countries_df['People_fully_vaccinated'].astype('Int64')


countries_df["Fully_vaccinated_percent"] = (countries_df["People_fully_vaccinated"]/ (countries_df["Population"]))*100
countries_df['Fully_vaccinated_percent'] = countries_df['Fully_vaccinated_percent'].astype('float64')
countries_df['Fully_vaccinated_percent']  = countries_df['Fully_vaccinated_percent'].apply(lambda x: round(x,2))
countries_df.loc[countries_df["Country_Region"] == "China"]
countries_df['sub-region'].unique()
countries_df
#countries_df.loc[countries_df["Country_Region"] == "US"]

In [ ]:
top10_table = countries_df.head(10)
top10_table = top10_table[["Country_Region","7_Days_Incidence_Rate"]]
top10_table = top10_table.rename(columns={'Country_Region': 'Country'})
top10_table 

In [ ]:
choro_vaccine_map = px.choropleth(countries_df, locations="Country_Region",
                    locationmode="country names",
                    color="Fully_vaccinated_percent",
                    template="plotly_dark",
                    projection="natural earth",
                    hover_name="Country_Region",
                    hover_data={
                                "Population": ":,",
                                "People_fully_vaccinated": ":,",
                                 "Country_Region": False
                            },
                    color_continuous_scale=px.colors.sequential.Oryel)

choro_vaccine_map.update_layout(
    margin=dict(l=0, r=0, t=50, b=50),
    title={
        'text': "% of the population fully vaccinated against COVID-19",
        'y':0.97,
        'x':0.424,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_font=dict(size=28))
choro_vaccine_map.show()

In [ ]:
import plotly.express as px

bubble_map = px.scatter_geo(countries_df,
                            size="7_Days_Incidence_Rate",
                            hover_name="Country_Region",
                            color="7_Days_Incidence_Rate",
                            locations="Country_Region",
                            locationmode="country names",
                            size_max=30,
                            template="plotly_dark",
                            color_continuous_scale=px.colors.sequential.dense,
                            projection="natural earth",
                            hover_data={
                                "Confirmed": ":,",
                                "Deaths": ":,",
                                "Population": ":,",
                                "Yesterday_Confirmed": ":,",
                                 "Country_Region": False
                            })
bubble_map.update_layout(
    margin=dict(l=0, r=0, t=50, b=50),
    title={
        'text': "7 Days COVID-19 Incidence Rate By Country",
        'y':0.97,
        'x':0.424,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_font=dict(size=28))
bubble_map

In [ ]:
#totals_df = daily_df[["Confirmed","Deaths"]].sum().reset_index(name="count")

totals_df = countries_df[["Confirmed","Deaths","People_fully_vaccinated"]].sum().reset_index(name="count")
totals_df = totals_df.rename(columns={'index':"condition"})

df_pop_sum = pd.DataFrame(
    {
        "condition": ["Population"],
        "count": [7794592576.0]
    }
)

totals_df = totals_df.append(df_pop_sum, sort=False) #7794592576.0
totals_df = totals_df.reset_index()
totals_df = totals_df.drop('index', 1)
totals_df['count'] =totals_df['count'].astype('Int64')
totals_df 
totals_df_bar = totals_df.iloc[2:4]


bars_graph = px.bar(totals_df_bar,
                    x="condition",
                    y="count",
                    template="plotly_dark",
                    hover_data={'count': ":,"},
                    labels={"condition": "Condition", "count": "Count", "color": "Condition"})
bars_graph.update_traces(marker_color=["#ff5e57", "#0be881"])
bars_graph.update_layout(
    margin=dict(l=0, r=0, t=50, b=50),
    title={
        'text': "Number of People Fully Vaccinated and World Population",
        'y':0.97,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    title_font=dict(size=28))
bars_graph

In [ ]:
vac_ti_url = 'https://github.com/govex/COVID-19/blob/master/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv?raw=true'
df_vaccine_time = pd.read_csv(vac_ti_url)
df_vaccine_time = df_vaccine_time[["Country_Region","Date","People_fully_vaccinated"]]
df_vaccine_time 

In [ ]:
vac_url = 'https://github.com/govex/COVID-19/blob/master/data_tables/vaccine_data/global_data/vaccine_data_global.csv?raw=true'
df_vaccine = pd.read_csv(vac_url)
df_vaccine = df_vaccine[(df_vaccine['Province_State'].isnull())]
df_vaccine = df_vaccine[["Country_Region","Date","People_fully_vaccinated"]]
df_vaccine_world = df_vaccine[df_vaccine["Country_Region"] == "World"]
df_vaccine_world = df_vaccine_world[["People_fully_vaccinated"]]
df_vaccine_world = df_vaccine_world.rename(columns={'People_fully_vaccinated': 'count'})
df_vaccine_world.insert(0, "condition", ["Fully vaccinated"], True)
df_vaccine_world

In [ ]:
daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{yesterday}.csv?raw=true'
daily_df = pd.read_csv(daily_url)
daily_df

In [ ]:
url_time_confirmed ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
time_confirmed_df = pd.read_csv(url_time_confirmed)
time_confirmed_df 
time_confirmed_df = time_confirmed_df.drop(["Province/State","Country/Region","Lat","Long"], axis =1).sum().reset_index(name="total")
time_confirmed_df

In [ ]:
daily_time_url ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
daily_time_df = pd.read_csv(daily_time_url)
daily_time_df = daily_time_df.drop(["Lat","Long"], axis =1)
daily_time_df= daily_time_df[(daily_time_df['Province/State'].isnull())]
daily_time_df = daily_time_df.drop(["Province/State"], axis =1)
daily_time_df

In [ ]:
def make_country_confirmed_df(country):
    url_time_confirmed = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
    df = pd.read_csv(url_time_confirmed)
    df = df.loc[df["Country/Region"] == country]
    df = df.drop(columns=["Province/State", "Country/Region",
                 "Lat", "Long"]).sum().reset_index(name="Confirmed")
    df = df.rename(columns={"index": "date"})
    return df
make_country_confirmed_df("Germany")

In [ ]:
def make_country_confirmed_df(country):
    url_time_confirmed = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
    df = pd.read_csv(url_time_confirmed)
    df = df.loc[df["Country/Region"] == country]
    df = df.drop(columns=["Province/State", "Country/Region",
                 "Lat", "Long"]).sum().reset_index(name="Confirmed")
    df = df.rename(columns={"index": "date"})
    return df

df = make_country_confirmed_df("Germany")

fig = px.line(
        df,
        x="date",
        y="Confirmed",
        template="plotly_dark",
        labels={"value": "Cases", "variable": "Confirmed", "date": "Date"},
        #hover_data={"value": ":,", "variable": False, "date": False},
    )

fig.show()

In [ ]:
daily_url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_daily_reports/{yesterday}.csv?raw=true'
daily_df = pd.read_csv(daily_url)
countries_df = daily_df[["Country_Region", "Confirmed", "Deaths"]]

countries_df = (countries_df.groupby("Country_Region").sum(
).sort_values(by="Confirmed", ascending=False).reset_index())


pop_url = 'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv?raw=true'
pop_df= pd.read_csv(pop_url)
pop_df= pop_df[(pop_df['Province_State'].isnull())]
pop_df = pop_df[["Country_Region","Population","iso3"]]
pop_df

continent_url = 'https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv?raw=true'
continent_df = pd.read_csv(continent_url)
continent_df = continent_df[["alpha-3","region","sub-region"]]
continent_df = continent_df.rename(columns={'alpha-3': 'iso3'})
pop_df = pd.merge(pop_df,continent_df, on="iso3")
countries_df = pd.merge(countries_df, pop_df, on="Country_Region")

daily_time_url ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
daily_time_df = pd.read_csv(daily_time_url)
daily_time_df = daily_time_df.drop(["Lat","Long"], axis =1)

daily_time_df_AuCaCh = daily_time_df.dropna(subset=['Province/State'])
daily_time_df_AuCaCh = daily_time_df_AuCaCh.groupby("Country/Region").sum()
daily_time_df_AuCaCh = daily_time_df_AuCaCh.loc[['Australia','Canada','China'], :]
daily_time_df_AuCaCh = daily_time_df_AuCaCh.reset_index()
daily_time_df= daily_time_df[(daily_time_df['Province/State'].isnull())]
daily_time_df = daily_time_df.drop(["Province/State"], axis =1)
daily_time_df = daily_time_df.append(daily_time_df_AuCaCh, ignore_index=True)

#########################################################################
will_be_merged_df = daily_time_df.rename(columns={'Country/Region': 'Country_Region'})
merged_df = pd.merge(countries_df, will_be_merged_df ,on="Country_Region", how='inner')
###################################################
new_df = countries_df.iloc[:,:7]
for i in range(15,len(merged_df.columns)):
    new_df[merged_df.columns[i]] = ((merged_df.iloc[:, i] - merged_df.iloc[:, i-7])/ (merged_df["Population"]/100000)).astype(int)
new_df
new_df = new_df.drop(['Confirmed', 'Deaths'], axis=1)
new_df
new_melted_df=new_df.melt(id_vars=["Country_Region", "region","sub-region","Population","iso3"], 
        var_name="Date", 
        value_name="7_Days_Incidence_Rate")

new_melted_df = new_melted_df.sort_values(by="Country_Region")
new_melted_df = new_melted_df.rename(columns={'7_Days_Incidence_Rate': 'Incidence_Rate'})
new_melted_df = new_melted_df.rename(columns={'sub-region': 'subregion'})
new_melted_df = new_melted_df.rename(columns={'Country_Region': 'Country'})
new_melted_df
new_melted_df["Date"] = pd.to_datetime(new_melted_df["Date"]) 
new_melted_df["Date"] = new_melted_df["Date"].dt.strftime('%Y-%m-%d')
new_melted_df=new_melted_df.sort_values(by=['Country','Date']).reset_index()
new_melted_df = new_melted_df.drop('index', 1)
for i in new_melted_df.index[new_melted_df['Incidence_Rate'] < 0]:
        new_melted_df['Incidence_Rate'].iloc[i] = new_melted_df['Incidence_Rate'].iloc[i-1]

#eu_vaccine_df['date_str'] = eu_vaccine_df['date'].apply(lambda x: str(x)) I explained the issue in more depth in a new question: 
new_melted_df 

In [ ]:
vac_owid_url ='https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations.csv?raw=true'
df_vac_owid_for_merge = pd.read_csv(vac_owid_url)
df_vac_owid_for_merge = df_vac_owid_for_merge.loc[df_vac_owid_for_merge["date"]>= '2021-05-01']
df_vac_owid_for_merge = df_vac_owid_for_merge[["location","iso_code","date","people_fully_vaccinated","total_boosters"]]
df_vac_owid_for_merge = df_vac_owid_for_merge.rename(columns={'iso_code': 'iso3'})
df_vac_owid_for_merge = df_vac_owid_for_merge.rename(columns={'date': 'Date'})
#df_vac_owid["Date"] = pd.to_datetime(df_vac_owid["Date"], format = '%Y-%m-%d') 
#df_vac_owid.groupby('location')['people_fully_vaccinated'].apply(lambda x : x.interpolate())
df_vac_owid_for_merge[["people_fully_vaccinated"]].isna().sum()
df_vac_owid_for_merge.info()
new_melted_sub_df = new_melted_df.loc[new_melted_df["Date"]>= '2021-05-01'] 
new_melted_sub_df.info()

In [ ]:

all_merged_df['Population'] =all_merged_df['Population'].astype(int)

In [ ]:
all_merged_df = new_melted_sub_df.merge(df_vac_owid, on=["iso3","Date"])
all_merged_df[["people_fully_vaccinated"]] = all_merged_df[["people_fully_vaccinated"]].fillna(method='ffill')
all_merged_df[["people_fully_vaccinated"]] = all_merged_df[["people_fully_vaccinated"]].fillna(method='bfill')
all_merged_df = all_merged_df.drop('location', 1)
all_merged_df["Fully_vaccinated_percent"] = (all_merged_df["people_fully_vaccinated"]/ (all_merged_df["Population"]))*100
all_merged_df
import plotly.express as px
df = px.data.gapminder()
animation_graph = px.scatter(all_merged_df, x="Fully_vaccinated_percent", y="Incidence_Rate", animation_frame="Date", animation_group="Country",
           size="Population", color="region", hover_name="Country",facet_col="region", 
           color_discrete_sequence=px.colors.sequential.Agsunset,
           log_y=True, 
           template="plotly_dark",
           size_max=40, range_x=[0,100], range_y=[10,1000])

animation_graph

In [ ]:
animation_graph = px.scatter(all_merged_df, x="Fully_vaccinated_percent", y="Incidence_Rate", 
                             #animation_frame="Date", 
                             #animation_group="Country",
           size="Population", color="region", hover_name="Country",facet_col="region", 
           color_discrete_sequence=px.colors.sequential.Agsunset,
           log_y=True, 
           template="plotly_dark",
           size_max=40, range_x=[0,100], range_y=[10,1000])

animation_graph

In [ ]:
new_melted_df["Incidence_Rate"].lt(0).sum()

In [ ]:
#NOT IMPORTANT
new_melted_df.index[new_melted_df['Incidence_Rate'] < 0]
new_melted_df.iloc[[3587]]
new_melted_df['Date'].iloc[[3587]] 
new_melted_df.index.isin([3587])
(new_melted_df['Incidence_Rate'] - (new_melted_df['Incidence_Rate'].iloc[3587])).abs().idxmin()

diff = (new_melted_df['Incidence_Rate'] - (new_melted_df['Incidence_Rate'].iloc[3587])).abs()
exclude = new_melted_df.index.isin([3587])
closer_idx = diff[~exclude].idxmin()
new_melted_df['Incidence_Rate'].iloc[closer_idx]
closer_idx

In [ ]:
#NOT IMPORTANT
new_melted_df["Incidence_Rate"].lt(0).sum()
minus = new_melted_df[new_melted_df["Incidence_Rate"] < 0]

minus.loc[minus["Country"] == "France"]
france = new_melted_df.loc[(new_melted_df['Country']=='France')]
france.loc[france['Date'] >= '2021-05-14'].head(20)
minus.head(38)
#minus.tail(40)

In [ ]:
#NOT IMPORTANT
import datetime
firstday = datetime.datetime.strptime("2020-1-30", '%Y-%m-%d')
new_melted_df['days']=firstday
new_melted_df['days']= (pd.to_datetime(new_melted_df['Date']) -  pd.to_datetime(new_melted_df['days'])).dt.days
new_melted_df
#pd.unique(new_melted_df["subregion"])
new_melted_df.info()

In [ ]:
import plotly.express as px
df = new_melted_df.query("subregion=='Western Europe'")
y_max = df.Incidence_Rate.max() 
fig = px.line(df, x="Date", y="Incidence_Rate", 
              #size="Incidence_Rate", 
              color="Country", 
              hover_name="Country",
              template="plotly_dark",
              #facet_col="region",
              #size_max=45, 
              #range_x=[0,685],
              range_y=[0,y_max])
fig.show()


In [14]:
new_melted_df.sort_values("subregion").subregion.unique()

array(['Australia and New Zealand', 'Central Asia', 'Eastern Asia',
       'Eastern Europe', 'Latin America and the Caribbean', 'Melanesia',
       'Micronesia', 'Northern Africa', 'Northern America',
       'Northern Europe', 'Polynesia', 'South-eastern Asia',
       'Southern Asia', 'Southern Europe', 'Sub-Saharan Africa',
       'Western Asia', 'Western Europe'], dtype=object)

In [20]:
unique = new_melted_df.sort_values("subregion").subregion.unique()
#all_continents = new_melted_df.subregion.unique()
#all_continents = all_continents.sort_values("subregion")
#all_merged_df
#if i is 
#new_melted_df['subregion_second'] = new_melted_df['subregion'].astype(str).str.split().str[1]
sorter = ['Eastern Asia', 'Central Asia', 'Southern Asia', 'South-eastern Asia', 'Western Asia', 
          'Northern Africa','Sub-Saharan Africa',
          'Latin America and the Caribbean', 'Northern America', 
          'Eastern Europe','Northern Europe','Southern Europe','Western Europe', 'Melanesia','Micronesia','Polynesia','Australia and New Zealand'] 
   
unique      
   

array(['Australia and New Zealand', 'Central Asia', 'Eastern Asia',
       'Eastern Europe', 'Latin America and the Caribbean', 'Melanesia',
       'Micronesia', 'Northern Africa', 'Northern America',
       'Northern Europe', 'Polynesia', 'South-eastern Asia',
       'Southern Asia', 'Southern Europe', 'Sub-Saharan Africa',
       'Western Asia', 'Western Europe'], dtype=object)

In [ ]:
fig = px.bar(
  new_melted_df.query("subregion=='Western Europe'").sort_values('days'), 
  x='Country', y='Incidence_Rate',
  color="Country",
  animation_frame='days',
  animation_group="Country",
  hover_name="Country",
  range_y=[0,1000]#,
  #range_x=[0,1000]
)
fig.update_layout(
  template='plotly_dark',
  margin=dict(r=10, t=25, b=40, l=60)
)
fig.show()

In [ ]:
url ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv?raw=true'
daily_df = pd.read_csv(url)
daily_df 
df = daily_df.drop(columns=["Province/State","Country/Region","Lat","Long"]).sum(skipna=True).reset_index(name="total")
df = df.rename(columns={'index':'date'})
df

In [ ]:
iso_url='https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv?raw=true'
df_iso_url = pd.read_csv(iso_url)
df_iso_url =df_iso_url[["Country","Alpha-3 code"]]
df_iso_url["Alpha-3 code"] = df_iso_url["Alpha-3 code"].str.replace('"', '')

df_iso_url = df_iso_url.rename(columns={'Country': 'Country_Region'})
df_iso_url = df_iso_url.rename(columns={'Alpha-3 code': 'code'})
countries_df_merged = pd.merge(countries_df, df_iso_url, on="Country_Region")
countries_df_merged
df_iso_url = pd.read_csv(iso_url)
df_iso_url

In [ ]:
conditions = ["confirmed", "deaths", "recovered"]
def make_global_df():
    def make_global_df(condition):
        url = f'https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_{condition}_global.csv?raw=true'
        df = pd.read_csv(url)
        df = df.drop(["Province/State","Country/Region","Lat","Long"], axis =1).sum().reset_index(name=condition)
        df = df.rename(columns={'index':'date'})
        return df

    final_df = None 
    for condition in conditions: 
        condition_df = make_global_df(condition)
        if final_df is None:
            final_df = condition_df
        else:
            final_df = final_df.merge(condition_df)
    return final_df
make_global_df()

In [ ]:
df = pd.read_csv("data/time_confirmed.csv")
df = df.loc[df["Country/Region"] == "Vietnam"]
df = df.drop(columns = ["Province/State","Country/Region", "Lat", "Long"]).sum().reset_index(name="confirmed")
df = df.rename(columns = {'index':"date"})
df

In [ ]:
conditions = ["confirmed", "deaths", "recovered"]

def make_country_df(country):
    def make_df(condition):
        df = pd.read_csv(f"data/time_{condition}.csv")
        df = df.loc[df["Country/Region"] == country]
        df = df.drop(columns = ["Province/State","Country/Region", "Lat", "Long"]).sum().reset_index(name=condition)
        df = df.rename(columns = {'index':"date"})
        return df

    final_df = None 

    for condition in conditions: 
        condition_df = make_df(condition)
        if final_df is None:
            final_df = condition_df
        else:
            final_df = final_df.merge(condition_df)
    return final_df  

df = make_country_df("Germany")
df

In [ ]:

def make_global_df():
    def make_df(condition)
        df = pd.read_csv(f"data/time_{condition}.csv")
        df = daily_df.drop(["Province/State", "Country/Region",
                           "Lat", "Long"], axis=1).sum().reset_index(name=condition)
        df = df.rename(columns={'index': 'date'})
        return df
    final_df = None
    for condition in conditions:
        condition_df = make_global_df(condition)
        if final_df is None:
            final_df = condition_df
        else:
            final_df = final_df.merge(condition_df)
    return final_df   

##

In [ ]:
daily_time_url ='https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
daily_time_df = pd.read_csv(daily_time_url)
daily_time_df = daily_time_df.drop(["Lat","Long"], axis =1)
daily_time_df_filled = daily_time_df.dropna(subset=['Province/State'])
daily_time_df_filled = daily_time_df_filled.groupby("Country/Region").sum()
daily_time_df_AuCaCh = daily_time_df_filled.loc[['Australia','Canada','China'], :]
daily_time_df_AuCaCh.reset_index()